## Social Media Campaign Generator Agent 

This notebook guides you through the steps of creating an AI Agent that will create targeted social media campaigns for hotels.

Execute each of the Python or SQL cells sequentially to generate the tools and AI Agent.

### Step 1: Create tools for AI Agent

You will create several functions to be used by the agent to help it generate the targeted social media posts.


In [0]:
catalog = "workspace"
db = "default"
warehouse_id = "66f2eab0f64875a4"

# Create a Databricks widget to store the catalog, database, and warehouse ID for all subsequent use
dbutils.widgets.text("catalog", catalog)
dbutils.widgets.text("database", db)
dbutils.widgets.text("warehouse_id", warehouse_id)

In [0]:
%sql
-- Pick any city from the result set of this query and store it as the city variable in the next cell

USE CATALOG IDENTIFIER(:catalog);
USE DATABASE IDENTIFIER(:database);
 
SELECT hc.CITY FROM hotel_catalog as hc
  JOIN hotel_reviews as hr
  -- Filter out cities that do not have any hotels with reviews
    ON hc.HOTEL_ID = hr.HOTEL_ID
GROUP BY hc.CITY;

CITY
Denver
Suez
Frankfurt
Paris
Naples
Bordeaux
Liverpool
Chennai
Melbourne
Lyon


In [0]:
# Set the city here from the one you selected from the previous cell

# EXAMPLE: 
# selected_city = 'London'

selected_city = 'Denver'

# Create a Databricks widget to store the selected city for later SQL use
dbutils.widgets.text("selected_city", selected_city)

In [0]:
%sql
-- This function takes a city as an input and performs an aggregation to return the total sales and number of bookings for each HOTEL_ID in that city

USE CATALOG IDENTIFIER(:catalog);
USE DATABASE IDENTIFIER(:database);

CREATE OR REPLACE FUNCTION hotel_sales_per_city (input_city STRING COMMENT 'City to be searched')
returns table(total_sales BIGINT, total_bookings BIGINT, hotel_id STRING, hotel_name STRING)
LANGUAGE SQL
COMMENT 'This function takes a city as an input, and this time does an aggregate to return the sales and number of bookings for each HOTEL_ID in the city'
return
  (SELECT SUM(PRICE) AS total_sales, count(*) AS total_bookings, hotel_id, hotel_name
    FROM hotel_bookings AS hb
      WHERE hb.CITY = input_city GROUP BY HOTEL_ID, HOTEL_NAME
      order by total_sales desc)

In [0]:
%sql
-- Test out the `hotel_sales_per_city` function

USE CATALOG IDENTIFIER(:catalog);
USE DATABASE IDENTIFIER(:database);

SELECT * 
  FROM hotel_sales_per_city(:selected_city);

total_sales,total_bookings,hotel_id,hotel_name
819,4,H10000538,River Bordeaux Inn
682,3,H10000803,River Bordeaux Lodge
570,3,H10000625,River Bordeaux Luxury Spa
535,3,H10000465,River Bordeaux Inn
435,2,H10000750,River Bordeaux Suites
420,2,H10000483,River Bordeaux Luxury Hotel


In [0]:
%sql
-- This query will list the hotels with the most reviews. The more reviews it has, the more context your AI Agent will have in generating a social media campaign for it

USE CATALOG IDENTIFIER(:catalog);
USE DATABASE IDENTIFIER(:database);

SELECT 
  SUM(1) as review_count, 
  AVG(hr.REVIEW_RATING) as average_review_rating,
  hr.HOTEL_ID 
FROM hotel_reviews as hr
  GROUP BY hr.HOTEL_ID
  ORDER BY review_count desc;

review_count,average_review_rating,HOTEL_ID
5,3.6,H10000582
5,4.0,H10000802
5,4.4,H10000772
4,4.25,H10000668
4,4.25,H10000480
4,4.0,H10000588
3,4.666666666666667,H10000706
3,4.0,H10000448
3,4.0,H10000603
3,4.666666666666667,H10000316


In [0]:
# Pick a hotel_id from the query result set in the previous cell and persist it here for use in the next cell

# EXAMPLE: 
# hotel_with_reviews = 'H10000538'

hotel_with_reviews = 'H10000802'

# Create a Databricks widget to store the selected hotel ID for later use in SQL
dbutils.widgets.text("hotel_with_reviews", hotel_with_reviews)

In [0]:
%sql
-- This function takes a HOTEL_ID as input and generates a summary of the top 3 reasons people like the specified hotel.

USE CATALOG IDENTIFIER(:catalog);
USE DATABASE IDENTIFIER(:database);

CREATE OR REPLACE FUNCTION
hotel_summarize_customer_reviews(input_hotel_id STRING COMMENT 'ID of the hotel to be searched')
returns STRING
LANGUAGE SQL
COMMENT 'This function will fetch the best feedback from a Hotel and summarize them'
return
  SELECT AI_GEN(SUBSTRING('Extract the top 3 reasons people like the hotel based on this list of review:' || ARRAY_JOIN(COLLECT_LIST(hr.REVIEW_TEXT), ' - '), 1, 80000)) AS all_reviews
  FROM hotel_reviews as hr
    WHERE hr.HOTEL_ID = input_hotel_id 
    AND hr.REVIEW_RATING >= 3

In [0]:
%sql
-- Try out this function to see if it provides a good summary of the customer reviews of a hotel

USE CATALOG IDENTIFIER(:catalog);
USE DATABASE IDENTIFIER(:database);

SELECT hotel_summarize_customer_reviews('${hotel_with_reviews}')

hotel_summarize_customer_reviews('H10000802')
"Based on the reviews, the top 3 reasons people like the hotel are: 1. **Ambience and Decor**: The hotel's enchanting ambience, grand lobby, and elegant decor create a perfect blend of sophistication and warmth, setting the tone for a memorable stay. 2. **Location**: The hotel's unbeatable location in the heart of the city, yet serene and peaceful, makes it incredibly convenient to explore the city's prime attractions. 3. **Staff and Service**: The exceptional staff, with their warm welcome, attentive service, professionalism, and genuine kindness, make a significant contribution to a remarkable stay. Note that the amenities (such as the fitness center and spa) are also mentioned as a positive aspect, but they seem to be a secondary reason compared to the top 3 mentioned above."


## Step 2: Create the AI Agent

In this step you are going to combine these three cruicial parts of our agent:

1. Tools for the Agent to use (from step 1)
2. LLM to serve as the agent's "brains"
3. System prompt that defines guidelines for the agent's tasks

In [0]:
%pip install -U databricks-sdk==0.39.0 langchain-community==0.2.16 langchain-openai==0.1.19 mlflow==2.19.0

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
from langchain_community.tools.databricks import UCFunctionToolkit
import pandas as pd

# Retrieve IDs from widget
warehouse_id = dbutils.widgets.get("warehouse_id")
catalog = dbutils.widgets.get("catalog")
db = dbutils.widgets.get("database")

print(warehouse_id)

def get_tools():
    return (
        UCFunctionToolkit(warehouse_id=warehouse_id)
        # Include functions as tools using their qualified names.
        .include(f"{catalog}.{db}.*")
        .get_tools())

In [0]:
from langchain_community.chat_models.databricks import ChatDatabricks

# We're going to use llama 3.3 because it's tool-enabled and is available. Keep temp at 0 to make it more deterministic.
llm = ChatDatabricks(endpoint="databricks-meta-llama-3-3-70b-instruct",
    temperature=0.0,
    streaming=False)

In [0]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatDatabricks

#This defines our agent's system prompt. Here we can tell it what we expect it to do and guide it on using specific functions. 

def get_prompt(history = [], prompt = None):
    if not prompt:
            prompt = """You are a helpful assistant for a global hotel company. Your task is to assist the marketing leadership in understanding their products and sales metrics. You can retrieve and analyze relevant data using specific functions:

        Use customer hotel reviews from the hotel_summarize_customer_reviews function to write a positive instagram post to promote the hotel in the given City that the lowest sales which you can find with the hotel_sales_per_city function. Mention the hotel by its name, but do not mention that the hotel is not performing well nor mention any of its flaws.

        Follow these guidelines:
        1. Call the appropriate function at each step and ensure results are retrieved before proceeding.
        2. Provide clear, coherent responses without mentioning the underlying functions.
        3. Answer only what the user asks for, no unnecessary information.
        4. If asked to generate Instagram posts, first determine what customers like most to ensure relevance.
        """
    return ChatPromptTemplate.from_messages([
            ("system", prompt),
            ("human", "{messages}"),
            ("placeholder", "{agent_scratchpad}"),
    ])

In [0]:
from langchain.agents import AgentExecutor, create_openai_tools_agent

prompt = get_prompt()
tools = get_tools()
agent = create_openai_tools_agent(llm, tools, prompt)

def model_parsing_error_handler(e):
    print(f"This error occurred during model parsing: {e}")

#Put the pieces together to create our Agent
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=model_parsing_error_handler)

In [0]:
from operator import itemgetter
from langchain.schema.runnable import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

# Very basic chain that allows us to pass the input (messages) into the Agent and collect the (output) as a string

agent_str = ({ "messages": itemgetter("messages")} | agent_executor | itemgetter("output") | StrOutputParser())


In [0]:
answer = ""

selected_city = dbutils.widgets.get("selected_city")

# try:
    
answer = agent_str.invoke({"messages": "Use customer hotel reviews to write a positive instagram post to promote the hotel in " + selected_city + " with the lowest sales. Mention the hotel by its name."})

# except Exception as e:
#     print(f"This error occurred during model invocation: {e}")

---------------------------------------------------------------------------
InvalidParameterValue                     Traceback (most recent call last)
File <command-1204291622731010>, line 7
      3 selected_city = dbutils.widgets.get("selected_city")
      5 # try:
----> 7 answer = agent_str.invoke({"messages": "Use customer hotel reviews to write a positive instagram post to promote the hotel in " + selected_city + " with the lowest sales. Mention the hotel by its name."})

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-d401c8bf-ef14-421d-bf4a-dda3ad423e0e/lib/python3.11/site-packages/langchain_core/runnables/base.py:2879, in RunnableSequence.invoke(self, input, config, **kwargs)
   2877             input = context.run(step.invoke, input, config, **kwargs)
   2878         else:
-> 2879             input = context.run(step.invoke, input, config)
   2880 # finish the root run
   2881 except BaseException as e:

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-d401c8bf-ef14-421d-bf4a-d

In [0]:
print(answer)